# 04 · Fine-tune Slice *t*

Configure LoRA/QLoRA adapters, derive gradient accumulation from the token budget, and simulate training loop accounting.

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")


In [ ]:
import math
from dataclasses import asdict

from src.modeling.lora import LoRAHyperParams, compute_gradient_accumulation, describe_setup
from src.modeling.train import TokenBudgetTracker, simulate_training_loop

# Placeholder hyper-parameters; update from configs/train_llm.yaml
lora_params = LoRAHyperParams(r=64, alpha=16, dropout=0.05, target_modules=('q_proj', 'k_proj'))
print(describe_setup('meta-llama/Llama-3.1-8B', load_in_4bit=True))

TOKENS_PER_STEP = 128_000
MICRO_BATCH = 1
AVG_TOKENS_PER_SAMPLE = 3_000
accum = compute_gradient_accumulation(TOKENS_PER_STEP, MICRO_BATCH, AVG_TOKENS_PER_SAMPLE)
print(f'Gradient accumulation: {accum}')

In [ ]:
# Simulated training loop using synthetic batches
tracker = TokenBudgetTracker(tokens_per_slice=25_000_000)
mock_batches = [(MICRO_BATCH, AVG_TOKENS_PER_SAMPLE) for _ in range(100_000)]
processed = simulate_training_loop(mock_batches, tracker)
print(f'Processed {processed} batches before hitting the token budget.')
print(f'Tokens consumed: {tracker.consumed_tokens:,}')
print(f'Remaining tokens: {tracker.remaining:,}')

🧪 **TODO:** Replace `simulate_training_loop` with an actual `transformers.Trainer` or `accelerate` loop, wiring in PEFT adapters and replay sampling.